In [191]:
import tensorflow as tf
import numpy as np

from matplotlib import animation, rc
import matplotlib_utils
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

tf.reset_default_graph()

s = tf.InteractiveSession()

from sklearn.datasets import load_digits
mnist = load_digits(2)

X, y = mnist.data, mnist.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

print("y [shape - %s]:" % (str(y.shape)), y[:10])
print("X [shape - %s]:" % (str(X.shape)))

with tf.name_scope("MSE"):
    y_true = tf.placeholder("float32", shape=(None,), name="y_true")
    y_predicted = tf.placeholder("float32", shape=(None,), name="y_predicted")
    
    mse = tf.reduce_mean((y_predicted - y_true)**2)

def compute_mse(vector1, vector2):
    return mse.eval({y_true: vector1, y_predicted: vector2})


weights = logreg_weights = tf.Variable(initial_value=tf.zeros((X.shape[1],1), dtype=tf.float32), name="weights")
b = logreg_bias = tf.Variable(initial_value=0., dtype="float32", name="bias")
input_X = logreg_input_x = tf.placeholder("float32", shape=(None,64), name="X")
input_y = logreg_input_y = tf.placeholder("float32", shape=(None,), name="y")

# print(logreg_input_x.shape)
# print(logreg_weights.shape)
matmul = tf.matmul(logreg_input_x, logreg_weights)
# print(matmul.shape)
squeezed = tf.squeeze(matmul)
# print(squeezed.shape)
# print(logreg_bias.shape)
preds = matmul + logreg_bias

# logreg_preds = tf.nn.sigmoid(tf.squeeze(tf.matmul(logreg_input_x, logreg_weights)) + logreg_bias)
logreg_preds = tf.squeeze(tf.nn.sigmoid(preds))

predicted_y = logreg_preds
# print(predicted_y.shape)
# print((predicted_y * logreg_input_y).shape)

# tmp_node = predicted_y * logreg_input_y

loss_a = -tf.log(predicted_y) * input_y
loss_b = tf.log(1 - predicted_y) * (1 - input_y)
loss_sum = loss_a - loss_b
loss = tf.reduce_mean(loss_sum)

l = -input_y*tf.log(predicted_y) - (1-input_y)*tf.log(1-predicted_y)

loss = tf.reduce_mean(l)

# loss = tf.reduce_mean(-tf.reduce_sum(input_y*tf.log(predicted_y)))
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=input_y))

# print(loss.shape)

optimizer = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.5) \
    .minimize(loss, var_list=[weights, b])

tf.summary.FileWriterCache.clear()
writer = tf.summary.FileWriter("/tmp/tboard", graph=loss.graph)
writer.flush()

my_scalar = tf.placeholder('float32')
my_vector = tf.placeholder('float32', [None])
# Compute the gradient of the next weird function over my_scalar and my_vector
# Warning! Trying to understand the meaning of that function may result in permanent brain damage
weird_psychotic_function = tf.reduce_mean(
    (my_vector+my_scalar)**(1+tf.nn.moments(my_vector,[0])[1]) + 
    1./ tf.atan(my_scalar))/(my_scalar**2 + 1) + 0.01*tf.sin(
    2*my_scalar**1.5)*(tf.reduce_sum(my_vector)* my_scalar**2
                      )*tf.exp((my_scalar-4)**2)/(
    1+tf.exp((my_scalar-4)**2))*(1.-(tf.exp(-(my_scalar-4)**2)
                                    )/(1+tf.exp(-(my_scalar-4)**2)))**2

der_by_scalar = tf.gradients(weird_psychotic_function, my_scalar)
der_by_vector = tf.gradients(weird_psychotic_function, my_vector)

validation_weights = 1e-3 * np.fromiter(map(lambda x:
        s.run(weird_psychotic_function, {my_scalar:x, my_vector:[1, 0.1, 2]}),
                                   0.15 * np.arange(1, X.shape[1] + 1)),
                                   count=X.shape[1], dtype=np.float32)[:, np.newaxis]
# Compute predictions for given weights and bias
prediction_validation = s.run(
    predicted_y, {
    input_X: X,
    weights: validation_weights,
    b: 1e-1})

# Load the reference values for the predictions
validation_true_values = np.loadtxt("validation_predictons.txt")

assert prediction_validation.shape == (X.shape[0],),\
       "Predictions must be a 1D array with length equal to the number " \
       "of examples in input_X"
assert np.allclose(validation_true_values, prediction_validation)

# print(prediction_validation)

# Compute predictions for given weights and bias
tmp_ = s.run(
    predicted_y, {
    input_X: X[:100],
    input_y: y[:100],
    weights: validation_weights + 1.21e-3,
    b: -1e-1})

print(prediction_validation[:10])
print(y[:10])

loss_validation = s.run(
        loss, {
            input_X: X[:100],
            input_y: y[-100:],
            weights: validation_weights+1.21e-3,
            b: -1e-1})
print(loss_validation)
assert np.allclose(loss_validation, 0.728689)

y [shape - (360,)]: [0 1 0 1 0 1 0 0 1 1]
X [shape - (360, 64)]:
[ 0.59420049  0.59367478  0.59943277  0.59905577  0.60508949  0.61222959
  0.6101138   0.60510671  0.58051741  0.58661669]
[0 1 0 1 0 1 0 0 1 1]
0.728689


In [195]:
from sklearn.metrics import roc_auc_score
s.run(tf.global_variables_initializer())
# weights.assign(validation_weights)
for i in range(5):
#     print(s.run(weights[:10]))
#     print("preds", s.run(preds, {input_X: X_train[:10], input_y: y_train[:10]}))
#     print("predicted_y", s.run(predicted_y, {input_X: X_train[:10], input_y: y_train[:10]}))
#     print("loss_a", s.run(loss_a, {input_X: X_train[:10], input_y: y_train[:10]}))
#     print("loss_a", s.run(loss_a, {input_X: X_train, input_y: y_train}))
#     print("loss_b", s.run(loss_b, {input_X: X_train[:10], input_y: y_train[:10]}))
#     print("loss", s.run(loss, {input_X: X_train[:10], input_y: y_train[:10]}))
#     print("y_train", y_train[:10])

    tmp =     s.run(optimizer, {input_X: X_train[:10], input_y: y_train[:10]})
#     print(s.run(weights[:10]))
    
    s.run(optimizer, {input_X: X_train, input_y: y_train})
#     print(s.run(weights[:10]))
    loss_i = s.run(loss, {input_X: X_train, input_y: y_train})
    print("loss at iter %i:%.4f" % (i, loss_i))
    print("train auc:", roc_auc_score(y_train, s.run(predicted_y, {input_X:X_train})))
    print("test auc:", roc_auc_score(y_test, s.run(predicted_y, {input_X:X_test})))

loss at iter 0:0.0554
train auc: 0.999945103206
test auc: 1.0
loss at iter 1:0.0257
train auc: 1.0
test auc: 1.0
loss at iter 2:0.0175
train auc: 1.0
test auc: 1.0
loss at iter 3:0.0148
train auc: 1.0
test auc: 1.0
loss at iter 4:0.0133
train auc: 1.0
test auc: 1.0


In [200]:
import sys
sys.path.append("..")
import grading



In [204]:
test_weights = 1e-3 * np.fromiter(map(lambda x:
    s.run(weird_psychotic_function, {my_scalar:x, my_vector:[1, 2, 3]}),
                               0.1 * np.arange(1, X.shape[1] + 1)),
                               count=X.shape[1], dtype=np.float32)[:, np.newaxis]

prediction_test = s.run(
    predicted_y, {
    input_X: X,
    weights: test_weights,
    b: 1e-1})

assert prediction_test.shape == (X.shape[0],),\
       "Predictions must be a 1D array with length equal to the number " \
       "of examples in X_test"
        
loss_test = s.run(
    loss, {
        input_X: X[:100],
        input_y: y[-100:],
        weights: test_weights+1.21e-3,
        b: -1e-1})
# Yes, the X/y indices mistmach is intentional

In [205]:
grade_submitter = grading.Grader("BJCiiY8sEeeCnhKCj4fcOA")

grade_submitter.set_answer("0ENlN", prediction_test)
grade_submitter.set_answer("D16Rc", roc_auc_score(y_test, s.run(predicted_y, {input_X:X_test})))

grade_submitter.submit('wesen@ruinwesen.com', '5rTVia02zPBGNfYn')

Your submission doesn't reference all parts of the assignment. Please make sure you have included references to all parts within your submission. Try downloading the latest starter files if the issue still persists.
